# SQL in Python + Pandas equivalents
This notebook shows how to run common SQL operations **from Python** (using SQLite) and then how to do the same operations using **pandas**.

Topics covered:
- `SELECT`
- `WHERE`
- `LIKE`
- `ORDER BY`
- `GROUP BY` + `COUNT`
- `JOIN`
- Side-by-side pandas equivalents (`query`, `str.contains`, `sort_values`, `groupby`, `merge`)


In [ ]:

# Setup: create sample data, load into SQLite, and keep DataFrames for pandas examples
import pandas as pd
import numpy as np
import sqlite3

customers = pd.DataFrame([
    {"customer_id": 1, "name": "Aisha", "city": "Cairo"},
    {"customer_id": 2, "name": "Omar", "city": "Riyadh"},
    {"customer_id": 3, "name": "Lina", "city": "Dubai"},
    {"customer_id": 4, "name": "Maya", "city": "Cairo"},
])

orders = pd.DataFrame([
    {"order_id": 101, "customer_id": 1, "product": "Laptop",  "amount": 1200, "order_date": "2025-01-03"},
    {"order_id": 102, "customer_id": 1, "product": "Mouse",   "amount": 25,   "order_date": "2025-01-05"},
    {"order_id": 103, "customer_id": 2, "product": "Monitor", "amount": 300,  "order_date": "2025-01-09"},
    {"order_id": 104, "customer_id": 3, "product": "Laptop",  "amount": 1100, "order_date": "2025-01-12"},
    {"order_id": 105, "customer_id": 3, "product": "Cable",   "amount": 10,   "order_date": "2025-01-13"},
    {"order_id": 106, "customer_id": 4, "product": "Laptop",  "amount": 1300, "order_date": "2025-01-20"},
])

# Create an in-memory SQLite database
con = sqlite3.connect(":memory:")

# Load DataFrames to SQL tables
customers.to_sql("customers", con, index=False, if_exists="replace")
orders.to_sql("orders", con, index=False, if_exists="replace")

customers, orders


## 1) SELECT
### SQL
Get all columns from a table.


In [ ]:

sql = "SELECT * FROM customers;"
pd.read_sql(sql, con)


### pandas equivalent
In pandas, you already have the DataFrame.


In [ ]:
customers.copy()

## 2) WHERE
### SQL
Filter rows (e.g., customers in Cairo).


In [ ]:

sql = "SELECT * FROM customers WHERE city = 'Cairo';"
pd.read_sql(sql, con)


### pandas equivalent

In [ ]:

customers[customers["city"] == "Cairo"]


## 3) LIKE
### SQL
`LIKE` is pattern matching. Here we find products that contain `lap` (case-insensitive depends on DB). In SQLite, `LIKE` is case-insensitive for ASCII by default.


In [ ]:

sql = "SELECT * FROM orders WHERE product LIKE '%lap%';"
pd.read_sql(sql, con)


### pandas equivalent
Use `str.contains` for substring/pattern matching.


In [ ]:

orders[orders["product"].str.contains("lap", case=False, na=False)]


## 4) ORDER BY
### SQL
Sort rows by a column.


In [ ]:

sql = "SELECT * FROM orders ORDER BY amount DESC;"
pd.read_sql(sql, con)


### pandas equivalent

In [ ]:

orders.sort_values("amount", ascending=False)


## 5) GROUP BY + COUNT
### SQL
Count how many orders each customer made.


In [ ]:

sql = '''
SELECT customer_id, COUNT(*) AS order_count
FROM orders
GROUP BY customer_id
ORDER BY order_count DESC;
'''
pd.read_sql(sql, con)


### pandas equivalent
Use `groupby` + `size()` (or `count()`).


In [ ]:

orders.groupby("customer_id").size().reset_index(name="order_count").sort_values("order_count", ascending=False)


## 6) JOIN
### SQL
Join `orders` with `customers` to get customer names & cities along with each order.


In [ ]:

sql = '''
SELECT
  o.order_id,
  o.product,
  o.amount,
  o.order_date,
  c.name,
  c.city
FROM orders o
JOIN customers c
  ON o.customer_id = c.customer_id
ORDER BY o.amount DESC;
'''
pd.read_sql(sql, con)


### pandas equivalent
Use `merge`.


In [ ]:

joined = orders.merge(customers, on="customer_id", how="inner")
joined.sort_values("amount", ascending=False)[["order_id","product","amount","order_date","name","city"]]


## 7) JOIN + WHERE + GROUP BY + COUNT (combined example)
### SQL
Example: count **Laptop** orders per city.


In [ ]:

sql = '''
SELECT
  c.city,
  COUNT(*) AS laptop_orders
FROM orders o
JOIN customers c
  ON o.customer_id = c.customer_id
WHERE o.product = 'Laptop'
GROUP BY c.city
ORDER BY laptop_orders DESC;
'''
pd.read_sql(sql, con)


### pandas equivalent
Filter → merge → groupby.


In [ ]:

(
    orders.loc[orders["product"] == "Laptop"]
    .merge(customers, on="customer_id", how="inner")
    .groupby("city")
    .size()
    .reset_index(name="laptop_orders")
    .sort_values("laptop_orders", ascending=False)
)


## 8) Extra: `WHERE` with multiple conditions
### SQL
Orders with amount >= 300 **and** product not equal to Cable.


In [ ]:

sql = '''
SELECT *
FROM orders
WHERE amount >= 300 AND product <> 'Cable'
ORDER BY amount DESC;
'''
pd.read_sql(sql, con)


### pandas equivalent
Use boolean masks (or `query`).


In [ ]:

orders[(orders["amount"] >= 300) & (orders["product"] != "Cable")].sort_values("amount", ascending=False)


## 9) Quick cheat sheet
- **SQL `WHERE`** → pandas boolean mask / `query`
- **SQL `LIKE '%x%'`** → `str.contains('x', case=False)`
- **SQL `ORDER BY col DESC`** → `sort_values('col', ascending=False)`
- **SQL `GROUP BY key` + `COUNT(*)`** → `groupby(key).size()`
- **SQL `JOIN`** → `merge` (usually on a key column)


## Tasks
For each task:
- Write a SQL solution (as a string) and run it with `pd.read_sql(sql, con)`
- Write the pandas equivalent below it

Try to solve without looking at the Answer Key first.


In [ ]:

# Setup: sample data + SQLite database
import pandas as pd
import numpy as np
import sqlite3
from itertools import combinations

customers = pd.DataFrame([
    {"customer_id": 1, "name": "Aisha", "city": "Cairo"},
    {"customer_id": 2, "name": "Omar", "city": "Riyadh"},
    {"customer_id": 3, "name": "Lina", "city": "Dubai"},
    {"customer_id": 4, "name": "Maya", "city": "Cairo"},
    {"customer_id": 5, "name": "Ziad", "city": "Dubai"},  # customer with no orders (for LEFT JOIN task)
])

orders = pd.DataFrame([
    {"order_id": 101, "customer_id": 1, "product": "Laptop",  "amount": 1200, "order_date": "2025-01-03"},
    {"order_id": 102, "customer_id": 1, "product": "Mouse",   "amount": 25,   "order_date": "2025-01-05"},
    {"order_id": 103, "customer_id": 2, "product": "Monitor", "amount": 300,  "order_date": "2025-01-09"},
    {"order_id": 104, "customer_id": 3, "product": "Laptop",  "amount": 1100, "order_date": "2025-01-12"},
    {"order_id": 105, "customer_id": 3, "product": "Cable",   "amount": 10,   "order_date": "2025-01-13"},
    {"order_id": 106, "customer_id": 4, "product": "Laptop",  "amount": 1300, "order_date": "2025-01-20"},
    {"order_id": 107, "customer_id": 4, "product": "Mouse",   "amount": 30,   "order_date": "2025-01-20"},
    {"order_id": 108, "customer_id": 4, "product": "Mouse",   "amount": 20,   "order_date": "2025-01-22"},
    {"order_id": 109, "customer_id": 2, "product": "Laptop",  "amount": 900,  "order_date": "2025-01-25"},
])

# Make order_date a real datetime in pandas (recommended)
orders["order_date"] = pd.to_datetime(orders["order_date"])

# SQLite in-memory DB
con = sqlite3.connect(":memory:")
customers.to_sql("customers", con, index=False, if_exists="replace")
# For SQLite, store dates as text ISO-8601; pandas ->_sql will convert datetime; we'll store as text explicitly:
orders_sql = orders.copy()
orders_sql["order_date"] = orders_sql["order_date"].dt.strftime("%Y-%m-%d")
orders_sql.to_sql("orders", con, index=False, if_exists="replace")

customers, orders.head()


### Task 1: Top Customers by Spending (Include Ties)

**Goal:**  
Find the **top 2 customers** by **total amount spent**, but **include ties**  
(e.g., if the customer in 2nd place is tied with others, include all tied customers).

---

**Expected Output:**  
- `customer_id`  
- `name`  
- `total_spent`  
- `order_count`

---

**Sorting:**  
- `total_spent` **DESC**
- then `order_count` **DESC**

---

#### SQL Hints
- Aggregate spending per customer using:
  - `SUM(amount)` → `total_spent`
  - `COUNT(*)` (or `COUNT(order_id)`) → `order_count`
- Use a window rank such as `DENSE_RANK()` to keep ties:
  - Rank customers by `total_spent DESC, order_count DESC`
  - Keep rows where rank `<= 2`
- If window functions aren’t available, use a **subquery** approach.

---

#### pandas Hints
- Use:
  - `groupby().agg(total_spent=("amount","sum"), order_count=("order_id","count"))`
- Sort by `total_spent` and `order_count` descending
- Use dense ranking:
  - `rank(method="dense")`
- Filter rank `<= 2` to include ties


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)


# pandas (your solution)
# TODO


### Task 2: Customers with Repeat Purchase Behavior

**Goal:**  
Identify customers who purchased the **same product at least 2 times**.

---

**Expected Output:**  
- `customer_id`  
- `name`  
- `product`  
- `times_bought`  

---

**Sorting:**  
- Sort results by `times_bought` in **descending order**

---

#### SQL Hints
- Join the `customers` and `orders` tables
- Group by:
  - `customer_id`
  - `product`
- Use:
  ```sql
  HAVING COUNT(*) >= 2


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)


# pandas (your solution)
# TODO


### Task 3: Revenue by City & Product + Share of City Revenue

**Goal:**  
For each **city**, calculate:
1) **Revenue per product** in that city  
2) The city’s **total revenue** (all products)  
3) The **percentage share** of each product’s revenue within the city  

---

**Expected Output:**  
- `city`  
- `product`  
- `revenue`  
- `city_total_revenue`  
- `revenue_share_pct`

---

**Formula:**  
- `revenue_share_pct = revenue / city_total_revenue * 100`

---

#### SQL Hints
- First **aggregate** revenue by `city + product`
- Then compute `city_total_revenue` using a **window sum** over each city:
  - `SUM(revenue) OVER (PARTITION BY city)`
- Compute percentage share from those two values

---

#### pandas Hints
- Start with:
  - `groupby(["city", "product"]).sum()` to get `revenue`
- Compute city totals with:
  - `groupby("city").sum()`
- `merge` product revenue with city totals
- Create `revenue_share_pct` as a new column


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)

# pandas (your solution)
# TODO

### Task 4: Latest Order per Customer (De-duplication)

**Goal:**  
Retrieve **each customer’s most recent order** based on `order_date`.  
If a customer has **multiple orders on the same date**, select the one with the **highest amount**.

---

**Expected Output:**  
- `customer_id`  
- `name`  
- `order_id`  
- `order_date`  
- `amount`  
- `product`  

---

#### SQL Hints
- Use a **window function**
- Apply:
  ```sql
  ROW_NUMBER() OVER (
      PARTITION BY customer_id
      ORDER BY order_date DESC, amount DESC
  )


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)

# pandas (your solution)
# TODO

### Task 5: Customers with No Orders (LEFT JOIN)

**Goal:**  
Show all customers and their order counts, including customers who have **0 orders**.

**Expected Output:**  
- `customer_id`  
- `name`  
- `order_count`  

**Sorting:**  
- Sort results by `order_count` in **ascending order**

---

#### SQL Hints
- Use a **LEFT JOIN** between `customers` and `orders`
- Use `COUNT(o.order_id)` to count orders
- Group by customer fields

---

#### pandas Hints
- Use `merge(how="left")`
- Use `groupby`
- Replace missing values (`NaN`) with `0`


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)

# pandas (your solution)
# TODO

### Task 6: Basket Analysis (Product Pairs)

**Goal:**  
Identify **pairs of products** that were bought by the **same customer** (across any number of orders).

This is a classic **market basket analysis** problem.

---

**Expected Output:**  
- `product_a`  
- `product_b`  
- `customers_count` (number of unique customers who bought both products)

---

**Rules:**  
- Only keep pairs where `product_a < product_b`  
  (avoids duplicates like `(Laptop, Mouse)` and `(Mouse, Laptop)`)
- Count **unique customers**, not total orders

---

#### SQL Hints
- Use a **self join** on the `orders` table:
  - `o1.customer_id = o2.customer_id`
  - `o1.product < o2.product`
- Use `COUNT(DISTINCT customer_id)` to count unique customers
- Group by both products

---

#### pandas Hints
- Get **unique products per customer**
- Generate product **combinations** for each customer
- Count how many **distinct customers** appear in each pair


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)

# pandas (your solution)
# TODO

### Task 7: Spending Segments

**Goal:**  
Assign each customer a **spending segment** based on their total amount spent.

---

**Segmentation Rules:**  
- **VIP** → `total_spent >= 1000`  
- **Regular** → `200 <= total_spent < 1000`  
- **Low** → `total_spent < 200`

---

**Expected Output:**  
- `customer_id`  
- `name`  
- `total_spent`  
- `segment`

---

#### SQL Hints
- First compute `total_spent` per customer using `SUM(amount)`
- Use a `CASE WHEN` statement to assign segments:
  - `VIP`
  - `Regular`
  - `Low`
- Consider a `LEFT JOIN` so customers with no orders get `total_spent = 0`

---

#### pandas Hints
- Aggregate spending per customer using `groupby`
- Assign segments using:
  - `np.select`, **or**
  - `pd.cut` with custom bins and labels
- Ensure missing values are treated as `0` before segmentation


In [ ]:

# SQL (your solution)
sql = '''
-- TODO
'''
# pd.read_sql(sql, con)

# pandas (your solution)
# TODO